https://facebook.github.io/prophet/docs/quick_start.html

In [8]:
# Python
import pandas as pd
from prophet import Prophet
import mlflow.prophet
from prophet.plot import plot_plotly, plot_components_plotly
from prophet.diagnostics import performance_metrics

from sklearn.metrics import mean_squared_error


## 데이터 불러오기

In [9]:
# Python
df = pd.read_csv('https://raw.githubusercontent.com/facebook/prophet/main/examples/example_wp_log_peyton_manning.csv')
df.head()

,ds,y
0,2007-12-10,9.590761
1,2007-12-11,8.519590
2,2007-12-12,8.183677
3,2007-12-13,8.072467
4,2007-12-14,7.893572


## 학습, 테스트셋 준비

In [10]:
df['ds'] = pd.to_datetime(df.ds)
df_train = df[df.ds < "2015-01-01"]
df_test = df[(df.ds >= "2015-01-01") & (df.ds < "2016-01-01")]

## 모델 학습 및 평가

In [16]:
experiment = mlflow.get_experiment_by_name("train_prophet_model")
if experiment is None:
    experiment_id = mlflow.create_experiment("train_prophet_model")
    print(f"Experiment {"train_prophet_model"} is created")
else:
    experiment_id = experiment.experiment_id

with mlflow.start_run() as run:
    model = Prophet()
    model.fit(df_train)

    future = model.make_future_dataframe(periods=365, include_history=False)
    forecast = model.predict(future)

    results = pd.merge(
        df_test[['ds','y']],
        forecast[['ds','yhat']],
        on='ds', how="left"
        )
    rmse_ = mean_squared_error(results.y, results.yhat)
    mlflow.log_metric("rmse", rmse_)
    mlflow.prophet.log_model(model, artifact_path="prophet_model")

SyntaxError: f-string: expecting '}' (1979133585.py, line 17)

## 시각화

In [17]:
future = m.make_future_dataframe(periods=365, include_history=True)
forecast = m.predict(future)
plot_plotly(m, forecast)

In [18]:
plot_components_plotly(m,forecast)

## 모델 등록

In [19]:
with mlflow.start_run(experiment_id=experiment_id) as run:
    mlflow.register_model(
            f"runs:/{run.info.run_id}/prophet_model", "prophet_model"
        )

/Users/jeanboy/workspace/time-series-ml-app/.venv/lib/python3.10/site-packages/_distutils_hack/__init__.py:11: UserWarning:

Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.

/Users/jeanboy/workspace/time-series-ml-app/.venv/lib/python3.10/site-packages/_distutils_hack/__init__.py:26: UserWarning:

Setuptools is replacing distutils.

Registered model 'prophet_model' already exists. Creating a new version of this model...
Created version '6' of model 'prophet_model'.
